In [174]:
from time import sleep
import Pyro5.api 
import matplotlib.pyplot as plt
from TimeTagger import *
import numpy as np
import pandas as pd

In [8]:
# Name of the file we want to read. 
# Remember, "Measurement.ttbin" files read all the files in sequence 'Measurement'
file = "C:/Users/HP/Downloads/TimeTags_RaulAir_2024-10-25_172331.1.ttbin"

In [9]:
fr = FileReader(file)
print("All channels available: {}".format(fr.getChannelList()))

print("Has data? ->", fr.hasData())
# Returns: True if more data is available for reading, False if all data has been read from all the files specified in the class constructor.

All channels available: (1, 2)
Has data? -> True


In [9]:
# Now we are going to print all the configuration of the TT-Ultra at the moment of the measurement
config = fr.getConfiguration()

for key, value in config.items():
    print(f"{key}: {value}")

channel number scheme: 2
configure version: 1
current time: 2024-09-19 11:57:14 +0200
devices: [{'FPGA ID': 11493476189726804, 'PCB UID': '00fdf287', 'PCB version': '1.8 (1)', 'minimum software version': '2.10.6', 'model': 'Time Tagger Ultra', 'serial': '2410001ABQ'}]
firmware version: TT-Ultra, FW6, TS 2024-02-09 12:21:10, OK 1.46
fpga link: {'enabled': False}
hardware buffer size: 67108864
inputs: [{'average rising falling': [False, False], 'channel': [1, -1], 'conditional filter filtered': [False, False], 'conditional filter triggers': [False, False], 'deadtime': [2000, 2000], 'delay hardware': [0, 0], 'delay software': [0, 0], 'event divider': [1, 1], 'hardware delay compensation': [333, 333], 'high priority channel': [False, False], 'input hysteresis': 0, 'input impedance high': False, 'input mux': 0, 'normalization': [True, True], 'resolution': 'Standard', 'resolution rms': 42.0, 'trigger level': 0.5}, {'average rising falling': [False, False], 'channel': [2, -2], 'conditional fi

### Some useful code for obtaining the timestamps on each channel

In [20]:
data = fr.getData(10)
# Reads the next n_events and returns the buffer object with the specified number of timetags.
print(fr.hasData())
# Returns: True if more data is available for reading, False if all data has been read from all the files specified in the class constructor.

print(data.getTimestamps())
# Returns: Event timestamps in picoseconds for all chosen channels.
print(data.getChannels())
# Returns: Channel number for each detected event.

print(data.tGetData)
# Returns: The data-stream time position when the TimeTagStream or FileWriter started data acquisition.
print(data.tStart)


data = fr.getData(4)
print(fr.hasData())
print(data.getTimestamps())
print(data.getChannels())

print(fr.getData(1).getTimestamps()-fr.getData(1).getTimestamps())

True
[87705510474222736 87705510528747397 87705510585374048 87705510597630751
 87705510624058831 87705510826691745 87705510873819933 87705511041779556
 87705511108139013 87705511428243454]
[1 1 2 2 2 1 1 1 1 2]
87705511495625968
87705510474222736
True
[87705511495625968 87705511612165209 87705511703644893 87705511817148361]
[2 2 2 1]
[-114106584]


In [53]:
virtual_tagger = createTimeTaggerVirtual()
virtual_tagger.getConfiguration()

# measure photon antibunching
corr_ch1 = 1 # first photon channel for antibunching measurements
corr_ch2 = 2 # second photon channel for antibunching measurements
bwcorr = 1000 # 1 ns
nbins = 1000

cr = Countrate(virtual_tagger, [corr_ch1, corr_ch2])

virtual_tagger.setReplaySpeed(-1.0) # Speed of -1.0 will replay the data as fast as possible
virtual_tagger.replay(file, begin = 0, duration = -1) # collect data until the end 
virtual_tagger.waitForCompletion()

print(f"Rate of counts channel {corr_ch1} (counts/sec) : {cr.getData()[0]}")
print(f"Rate of counts channel {corr_ch2} (counts/sec) : {cr.getData()[1]}")

Rate of counts channel 1 (counts/sec) : 8003.999108212611
Rate of counts channel 2 (counts/sec) : 4305.408941877971


In [120]:
fr = FileReader(file)

acc_list = []
n = 10000
treshold = min(cr.getData())*1e-12
print(treshold, "\n")
t_ant = 0

while (fr.hasData()):
    data = fr.getData(n)
    
    ts = data.getTimestamps()
    ch = data.getChannels()
    
    data_raw = np.array([ch, ts])
    
    # Restructure into columns (one for each channel)
    data_ch1 = data_raw[1, np.where(data_raw[0,:] == 1)].copy() # Canal 1
    data_ch2 = data_raw[1, np.where(data_raw[0,:] == 2)].copy() # Canal 2
    
    # Creating an empty array the size of the largest dataset
    data_ch = np.zeros([2, max(data_ch1.shape[1], data_ch2.shape[1])])
    
    # Filling the channel columns with the respective data (one of them will always fall short)
    data_ch[0, :data_ch1.shape[1]] += data_ch1[0]
    data_ch[1, :data_ch2.shape[1]] += data_ch2[0]
    
    # Check we get all data
    #print(data_ch1.size, data_ch2.size, data_ch1.size+data_ch2.size)
    
    delta_t = abs(ts[-1] - ts[0])
    rate = n / delta_t
    print(rate)
    
    if (rate > treshold):
        acc_list.append(data_ch - t_ant)
        print("guardem")
    else:
        t_ant += delta_t
        print("no guardem")

4.3054089418779715e-09 

8.632483525735885e-09
guardem
8.483820351887072e-09
guardem
1.3595808130275222e-08
guardem
1.0559838770486288e-08
guardem
9.799524086369689e-09
guardem
7.729082891827796e-09
guardem
8.339618425505217e-09
guardem
7.375033117778058e-09
guardem
7.38034148975988e-09
guardem
8.128858000775838e-09
guardem
1.1201323485908385e-08
guardem
1.0563255194315016e-08
guardem
9.367976923909268e-09
guardem
1.0206768036596712e-08
guardem
1.2781858472037352e-08
guardem
1.3062881816158632e-08
guardem
6.8381195653992375e-09
guardem
7.489879569973469e-09
guardem
7.446806645977719e-09
guardem
6.4634664485918105e-09
guardem
6.120433878184063e-09
guardem
7.165709052216093e-09
guardem
8.2282264891381e-09
guardem
9.445289601458947e-09
guardem
8.259486821282495e-09
guardem
6.464948879656504e-09
guardem
6.492077627998406e-09
guardem
6.6139830498265966e-09
guardem
7.954843019599183e-09
guardem
1.077218712308551e-08
guardem
9.51617989336402e-09
guardem
7.586746344390086e-09
guardem
8.2181378

## Convert back into numpy array

In [175]:
total_len = sum(len(acc_list[i][0]) for i in range(len(acc_list)))
acc = np.zeros([total_len, 2])
index = 0

for i in range(len(acc_list)):
    for j in range(len(acc_list[i][0])):
        acc[index,:] = [acc_list[i][0][j], acc_list[i][1][j]]
        index += 1
        
df = pd.DataFrame(acc)

In [176]:
df

,0,1
0,8.770550e+16,8.770550e+16
1,8.770550e+16,8.770550e+16
2,8.770550e+16,8.770550e+16
3,8.770550e+16,8.770550e+16
4,8.770550e+16,8.770550e+16
...,...,...
19357519,9.012398e+16,0.000000e+00
19357520,9.012398e+16,0.000000e+00
19357521,9.012398e+16,0.000000e+00
19357522,9.012398e+16,0.000000e+00
